In [21]:
import requests

def fromNameToLatLon(names: list) -> dict:
    mapping = {}
    one_map_url = 'https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&searchVal='
    for location in names:
        request = one_map_url + location
        try: 
            response = requests.get(request).json()
            mapping[location] = (float(response['results'][0]['LATITUDE']), float(response['results'][0]['LONGITUDE']))
        except:
            print(f'Location: {location} not found in One Map')
            pass
    return mapping

In [22]:
import h3
import folium
import numpy as np
from scipy.spatial.distance import euclidean

def gaussian_probability(distance, sigma=0.01):
    """Compute the probability based on Gaussian distribution."""
    return np.exp(-distance**2 / (2 * sigma**2))

def get_hexagons_around_point(lat, lng, resolution=9, radius=0.03):
    """Generate H3 hexagons around a given point within a specified radius."""
    center_hex = h3.geo_to_h3(lat, lng, resolution)
    hexagons = set([center_hex])
    for k in range(1, 20):  # Increase this range for a wider search area
        hexagons.update(h3.k_ring(center_hex, k))
    # Filtering hexagons based on the specified radius
    hexagons = {hex for hex in hexagons if euclidean(h3.h3_to_geo(center_hex), h3.h3_to_geo(hex)) <= radius}
    return hexagons

def visualize_heatmap(points, resolution=9, sigma=0.01, threshold=0.1):
    base_map = folium.Map(location=points[0], zoom_start=14, tiles='cartodb positron')
    all_hexagons = set()

    for lat, lng in points:
        hexagons = get_hexagons_around_point(lat, lng, resolution)
        center_hex = h3.geo_to_h3(lat, lng, resolution)

        for index, hexagon in enumerate(hexagons):
            distance = euclidean(h3.h3_to_geo(center_hex), h3.h3_to_geo(hexagon))
            probability = gaussian_probability(distance, sigma)

            if probability > threshold:
                all_hexagons.add(hexagon)
                color_intensity = int(probability * 255)
                color = f"#{color_intensity:02x}0000"  # Red color with intensity based on probability
                polygon = h3.h3_to_geo_boundary(hexagon)
                # polygon = [(lat, lng) for (lng, lat) in polygon]
                folium.Polygon(locations=polygon, color=color, fill=True, fill_color=color, fill_opacity=0.6).add_to(base_map)
                folium.map.Marker(h3.h3_to_geo(hexagon),
                    icon=folium.DivIcon(
                        icon_size=(10,10),
                        icon_anchor=(5,14),
                        html=f'<div style="font-size: 8pt">{index}</div>'
                    )
                ).add_to(base_map)

    return base_map

In [25]:
name_hot_spots = ['SUTD', 'Tampines Secondary School']
mapping = fromNameToLatLon(name_hot_spots)
m = visualize_heatmap(list(mapping.values()))
display(m)